In [8]:
!pip install datasets fsspec gcsfs -q

In [9]:
from datasets import load_dataset

In [11]:
dataset = load_dataset("taln-ls2n/kp20k")

The repository for taln-ls2n/kp20k contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/taln-ls2n/kp20k.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] Y
BuilderConfig(name='raw', version=0.0.1, data_dir=None, data_files=None, description='This part of my dataset covers the raw data')


test.json:   0%|          | 0.00/25.3M [00:00<?, ?B/s]

train.json:   0%|          | 0.00/670M [00:00<?, ?B/s]

validation.json:   0%|          | 0.00/25.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/530809 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [12]:
# Selecting a subset of the train dataset
sample_percentage=0.25

dataset['train'] = dataset['train'].shuffle()
num_samples = int(len(dataset['train']) * sample_percentage)
train_set = dataset['train'].select(range(num_samples))

In [13]:
import torch
from torch.utils.data import Dataset

In [14]:
class HuggingFaceDataset(Dataset):
    def __init__(self,dataset):
        self.dataset=dataset
    def __getitem__(self,index):
        item=self.dataset[index]
        # ID=item['id']
        # title=item['title']
        # abstract=item['abstract']
        # keyphrases=item['keyphrases']
        # return ID,title,abstract,keyphrases
        return item
    def __len__(self):
        return len(self.dataset)

In [15]:
kp20k=HuggingFaceDataset(train_set)

In [16]:
from torch.utils.data import DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [17]:
class kp_data(HuggingFaceDataset):

    def __init__(self,dataset,tokenizer):
        super().__init__(dataset)
        self.tokenizer=tokenizer

    def __getitem__(self,idx):
        item=super().__getitem__(idx)
        abstract=item['abstract']
        keyphrases=item['keyphrases']

        input_text=f'detect keyword: abstract:{abstract}'

        target_text=f'{", ".join(keyphrases)}'

        input_ids=self.tokenizer.encode(input_text, truncation=True, padding='max_length', max_length=512, return_tensors='pt')[0]
        target_ids=self.tokenizer.encode(target_text,truncation=True, padding='max_length', max_length=32, return_tensors='pt')[0]

        return {'input_ids': input_ids, 'attention_mask': input_ids.ne(0), 'target_ids':target_ids, 'target_attention_mask': target_ids.ne(0)}


In [18]:
model_name='t5-base'

tokenizer=T5Tokenizer.from_pretrained(model_name)
kp_dataset=kp_data(train_set,tokenizer)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [33]:
model=T5ForConditionalGeneration.from_pretrained(model_name)
epochs=5
batch_size=8
learning_rate=2e-5

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
# optimizer=AdamW(model.parameters(),lr=learning_rate)
# scheduler=torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [20]:
from tqdm import tqdm
from time import time

In [24]:
max_source_length = 512
max_target_length = 128

In [28]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'abstract', 'keyphrases', 'prmu'],
        num_rows: 530809
    })
    test: Dataset({
        features: ['id', 'title', 'abstract', 'keyphrases', 'prmu'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['id', 'title', 'abstract', 'keyphrases', 'prmu'],
        num_rows: 20000
    })
})

In [29]:
train_dataset = kp_data(dataset['train'],tokenizer)

In [30]:
test_dataset = kp_data(dataset['test'],tokenizer)

In [31]:
train_dataset

In [ ]:
# train_dataloader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)


In [21]:
from transformers import Trainer,TrainingArguments

In [34]:
from transformers import DataCollatorForSeq2Seq

label_pad_token_id = -100
dataCollator = DataCollatorForSeq2Seq(
    tokenizer = tokenizer,
    model = model,
    label_pad_token_id = label_pad_token_id,
    pad_to_multiple_of = 8,
)

In [35]:
training_args = TrainingArguments(
    output_dir = './t5-mod/',
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    fp16 = False,
    learning_rate = 2e-5,
    num_train_epochs = 5,
    logging_strategy = "steps",
    logging_steps = 500,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    save_total_limit = 2,
    load_best_model_at_end= True,
    push_to_hub = False
)

trainer = Trainer(
    model = model,
    data_collator = dataCollator,
    train_dataset = train_dataset,
    eval_dataset = test_dataset
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [38]:
input_ids = tokenizer("""
Mitochondria are double-membraned organelles found in most eukaryotic cells. They are often referred to as the "powerhouses" of the cell because they generate most of the cell's energy in the form of adenosine triphosphate (ATP). Mitochondria play a crucial role in cellular respiration, which is the process by which cells convert nutrients into usable energy.
The structure of mitochondria consists of an outer membrane, which surrounds the entire organelle, and an inner membrane that is highly folded to form structures called cristae. The inner membrane encloses the mitochondrial matrix, which contains enzymes and DNA molecules necessary for various metabolic reactions.
One of the primary functions of mitochondria is to carry out aerobic respiration, a process that uses oxygen to break down glucose and other organic molecules, releasing energy in the form of ATP. This process occurs in the inner membrane of the mitochondria, specifically in the electron transport chain and the citric acid cycle.
Apart from energy production, mitochondria have other important roles in the cell. They are involved in the regulation of cellular metabolism, calcium signaling, and apoptosis (programmed cell death). Mitochondria also contain their own DNA, known as mitochondrial DNA (mtDNA), which is separate from the nuclear DNA found in the cell's nucleus.

It's worth noting that while mitochondria are present in most eukaryotic cells, certain cell types may have varying numbers of mitochondria depending on their energy requirements. For example, muscle cells and liver cells often contain a higher number of mitochondria due to their high energy demands.

""",return_tensors="pt").to(device).input_ids
output = model.generate(input_ids,max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))

the mitochondria are also the "powerhouses" of the cell. They are responsible for energy production and metabolism. Mitochondria are double-membraned organelles found in most eukaryotic cells. They are also involved in the regulation of cellular metabolism and apoptosis (programmed cell death). Mitochondria are double-membraned organelles that are found in most eukaryotic cells.
